In [1]:
pip install requests pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import requests
import pandas as pd
import os
import time
import json
from datetime import datetime
from typing import List, Dict, Any, Optional

SNAPSHOT_URL = "https://hub.snapshot.org/graphql"

# ==========================
# ✅ Uniswap smoke test config
# ==========================
spaces = ["uniswapgovernance.eth"]

OUT_CSV = "TEST_uniswap_votes.csv"
PROGRESS_JSON = "TEST_uniswap_progress.json"

PROPOSAL_BATCH = 100
VOTES_PAGE_SIZE = 1000

SLEEP_BETWEEN_CALLS = 0.8   # 如果 429 多，就调到 1.2~1.5
MAX_RETRIES = 5
BACKOFF_BASE = 2.0

SMOKE_TEST_MAX_PROPOSALS = None  # ✅ 只跑前 3 个 proposal；通过后改成 None 或删掉限制
# ==========================


def post_gql(query: str, variables: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """GraphQL POST with retry + exponential backoff + better error visibility."""
    payload = {"query": query}
    if variables:
        payload["variables"] = variables

    last_err = None
    for attempt in range(MAX_RETRIES):
        try:
            r = requests.post(SNAPSHOT_URL, json=payload, timeout=30)

            # ✅ 关键：先把 400 的内容打印出来（只在失败时）
            if not r.ok:
                print("\n========== SNAPSHOT HTTP ERROR ==========")
                print("Status:", r.status_code)
                print("Response text (first 800 chars):", r.text[:800])
                print("Query (first 800 chars):", query[:800])
                print("========================================\n")

            # 对 429/5xx 才重试；400 属于 query 错误，一般不该重试
            if r.status_code in (429, 500, 502, 503, 504):
                raise RuntimeError(f"HTTP {r.status_code}: {r.text[:200]}")

            r.raise_for_status()
            data = r.json()

            if "errors" in data:
                # GraphQL errors：也打印出来便于定位
                print("\n========== SNAPSHOT GQL ERROR ==========")
                print(str(data["errors"])[:800])
                print("=======================================\n")
                raise RuntimeError(f"GQL errors: {str(data['errors'])[:200]}")

            return data["data"]

        except Exception as e:
            last_err = e

            # ✅ 400 这种一般是 query 逻辑错误：直接抛出，不要浪费重试
            if isinstance(e, requests.HTTPError) and r is not None and r.status_code == 400:
                raise RuntimeError(f"❌ 400 Bad Request（GraphQL query 不合法）: {r.text[:500]}")

            wait = (BACKOFF_BASE ** attempt) + 0.1 * attempt
            print(f"⚠️ 请求失败（attempt {attempt+1}/{MAX_RETRIES}）：{e} | backoff {wait:.1f}s")
            time.sleep(wait)

    raise RuntimeError(f"❌ 多次重试仍失败：{last_err}")



def load_progress() -> set:
    if os.path.exists(PROGRESS_JSON):
        with open(PROGRESS_JSON, "r", encoding="utf-8") as f:
            return set(json.load(f))
    return set()


def save_progress(completed: set):
    with open(PROGRESS_JSON, "w", encoding="utf-8") as f:
        json.dump(sorted(list(completed)), f, ensure_ascii=False, indent=2)


def fetch_all_proposals(space: str) -> List[Dict[str, Any]]:
    """Paginate proposals for a space."""
    all_props = []
    skip = 0

    print(f"\n🚀 开始抓取 Space={space} 的全部 proposals ...")
    while True:
        q = f"""
        {{
          proposals(first: {PROPOSAL_BATCH}, skip: {skip},
            where: {{ space_in: ["{space}"] }},
            orderBy: "created", orderDirection: desc
          ) {{
            id
            title
            body
            choices
            author
            state
            created
          }}
        }}
        """
        data = post_gql(q)
        batch = data.get("proposals", [])
        print(f"📦 {space} proposals: {skip + 1} ~ {skip + len(batch)}")

        if not batch:
            break

        all_props.extend(batch)
        if len(batch) < PROPOSAL_BATCH:
            break

        skip += PROPOSAL_BATCH
        time.sleep(SLEEP_BETWEEN_CALLS)

    print(f"✅ {space} proposals 总数：{len(all_props)}")
    return all_props


def fetch_all_votes_for_proposal(proposal_id: str) -> List[Dict[str, Any]]:
    """
    Cursor pagination via created_lt to bypass skip<=5000 limit.
    created is a 32-bit Int (Unix seconds), so created_lt must be <= 2,147,483,647.
    """
    all_votes = []
    created_lt = int(time.time()) + 10  # ✅ 合法范围内的“未来一点点”
    seen = set()

    while True:
        q = f"""
        {{
          votes(first: {VOTES_PAGE_SIZE},
            where: {{ proposal: "{proposal_id}", created_lt: {created_lt} }},
            orderBy: "created", orderDirection: desc
          ) {{
            voter
            choice
            vp
            created
          }}
        }}
        """
        data = post_gql(q)
        batch = data.get("votes", [])

        if not batch:
            break

        added = 0
        for v in batch:
            key = (v.get("voter"), v.get("created"))
            if key in seen:
                continue
            seen.add(key)
            all_votes.append(v)
            added += 1

        # ✅ 方便你观察是否分页在继续
        print(f"   ↳ fetched {added} votes | total={len(all_votes)} | next created_lt={batch[-1]['created']}")

        last_created = batch[-1]["created"]

        # 防止死循环（保险）
        if last_created >= created_lt:
            break

        created_lt = last_created

        if len(batch) < VOTES_PAGE_SIZE:
            break

        time.sleep(SLEEP_BETWEEN_CALLS)

    return all_votes



def compute_majority_choice(votes: List[Dict[str, Any]]) -> Optional[int]:
    """Return the most common choice index (int). Supports int or dict choice."""
    counter: Dict[int, int] = {}
    for v in votes:
        c = v.get("choice")
        if isinstance(c, int):
            counter[c] = counter.get(c, 0) + 1
        elif isinstance(c, dict):
            for k, val in c.items():
                try:
                    kk = int(k)
                except:
                    continue
                if val and val > 0:
                    counter[kk] = counter.get(kk, 0) + 1

    if not counter:
        return None
    return max(counter.items(), key=lambda x: x[1])[0]


def append_rows_to_csv(rows: List[Dict[str, Any]]):
    """Append rows to CSV incrementally."""
    if not rows:
        return
    df = pd.DataFrame(rows)
    header = not os.path.exists(OUT_CSV)
    df.to_csv(OUT_CSV, mode="a", index=False, header=header)


def main():
    completed = load_progress()
    print(f"📌 已完成 proposals 数：{len(completed)}（来自 {PROGRESS_JSON}）")

    for space in spaces:
        proposals = fetch_all_proposals(space)

        for i, p in enumerate(proposals, 1):
            if SMOKE_TEST_MAX_PROPOSALS is not None and i > SMOKE_TEST_MAX_PROPOSALS:
                print(f"\n🧪 Smoke test 到此结束：已测试前 {SMOKE_TEST_MAX_PROPOSALS} 个 proposals。")
                return

            pid = p["id"]
            if pid in completed:
                print(f"⏭️ 跳过已完成 proposal: {p.get('title','')[:60]}")
                continue

            print(f"\n🔍 [{space}] Proposal {i}/{len(proposals)}：{p.get('title','')[:80]}")

            # 1) 抓全量 votes（分页）
            votes = fetch_all_votes_for_proposal(pid)
            print(f"🧾 votes 数：{len(votes)}")

            if not votes:
                completed.add(pid)
                save_progress(completed)
                continue

            total_vp = sum(v.get("vp", 0) for v in votes)
            if total_vp == 0:
                completed.add(pid)
                save_progress(completed)
                continue

            majority = compute_majority_choice(votes)
            if majority is None:
                completed.add(pid)
                save_progress(completed)
                continue

            # 2) 生成记录（保留你原来的字段逻辑）
            rows = []
            for v in votes:
                vp = v.get("vp", 0)
                vp_ratio = vp / total_vp if total_vp else 0

                choice_used = None
                aligned = False

                c = v.get("choice")
                if isinstance(c, int):
                    choice_used = c
                    aligned = (c == majority)
                elif isinstance(c, dict):
                    choice_used = json.dumps(c, ensure_ascii=False)
                    aligned = (str(majority) in c and c[str(majority)] > 0)

                rows.append({
                    "Space": space,
                    "Proposal ID": pid,
                    "Proposal Title": p.get("title", ""),
                    "Proposal Body": (p.get("body") or "").replace("\n", " ").replace("\r", " ")[:1000],
                    "Created Time": datetime.utcfromtimestamp(p["created"]).isoformat(),
                    "Voter": v.get("voter"),
                    "Choice": choice_used,
                    "Voting Power": vp,
                    "VP Ratio (%)": round(vp_ratio * 100, 4),
                    "Is Whale": vp_ratio > 0.05,
                    "Aligned With Majority": aligned,
                    "Vote Timestamp": datetime.utcfromtimestamp(v["created"]).isoformat()
                })

            # 3) 增量写入 CSV + 更新进度
            append_rows_to_csv(rows)
            completed.add(pid)
            save_progress(completed)

            print(f"✅ 已写入 {len(rows)} 行 | 累计完成 proposals={len(completed)}")
            time.sleep(SLEEP_BETWEEN_CALLS)

    print(f"\n🎉 测试完成！输出文件：{OUT_CSV}")


if __name__ == "__main__":
    main()


📌 已完成 proposals 数：102（来自 TEST_uniswap_progress.json）

🚀 开始抓取 Space=uniswapgovernance.eth 的全部 proposals ...
📦 uniswapgovernance.eth proposals: 1 ~ 100
📦 uniswapgovernance.eth proposals: 101 ~ 189
✅ uniswapgovernance.eth proposals 总数：189
⏭️ 跳过已完成 proposal: Strategic Renewal of Gnosis, Linea, and Mantle Deployments
⏭️ 跳过已完成 proposal: UNIfication
⏭️ 跳过已完成 proposal: Uniswap Community Proposal Factory
⏭️ 跳过已完成 proposal: Grow Uniswap on Plasma
⏭️ 跳过已完成 proposal:  Treasury Delegation Round 2 Elections
⏭️ 跳过已完成 proposal: GLI — Incentivized Delegation Vaults
⏭️ 跳过已完成 proposal: GLI — Treasury delegation Round 2
⏭️ 跳过已完成 proposal: Launching Uniswap v3 on Ronin with co-incentives
⏭️ 跳过已完成 proposal: Establish Uniswap Governance as “DUNI,” a Wyoming DUNA
⏭️ 跳过已完成 proposal: Uniswap Delegate Reward Initiative - Cycle 4
⏭️ 跳过已完成 proposal: [TEMP CHECK] - Etherlink Co-incentives Proposal
⏭️ 跳过已完成 proposal: Uniswap Unichain - USDS and sUSDS Co-Incentives Growth Manag
⏭️ 跳过已完成 proposal: Trial run a Technica

   ↳ fetched 1000 votes | total=9000 | next created_lt=1680139374
   ↳ fetched 445 votes | total=9445 | next created_lt=1680110970
🧾 votes 数：9445
✅ 已写入 9445 行 | 累计完成 proposals=104

🔍 [uniswapgovernance.eth] Proposal 105/189：[Temperature Check] Deploy Uniswap V3 on Gnosis Chain
   ↳ fetched 1000 votes | total=1000 | next created_lt=1679978538
   ↳ fetched 1000 votes | total=2000 | next created_lt=1679900141
   ↳ fetched 1000 votes | total=3000 | next created_lt=1679836700
   ↳ fetched 1000 votes | total=4000 | next created_lt=1679779822
   ↳ fetched 1000 votes | total=5000 | next created_lt=1679746404
   ↳ fetched 1000 votes | total=6000 | next created_lt=1679720786
   ↳ fetched 702 votes | total=6702 | next created_lt=1679689950
🧾 votes 数：6702
✅ 已写入 6702 行 | 累计完成 proposals=105

🔍 [uniswapgovernance.eth] Proposal 106/189：[Temperature Check] Deploy Uniswap v3 onto Kava EVM Network
   ↳ fetched 1000 votes | total=1000 | next created_lt=1678597021
   ↳ fetched 1000 votes | total=2000 | nex

   ↳ fetched 1000 votes | total=6000 | next created_lt=1671113744
   ↳ fetched 1000 votes | total=7000 | next created_lt=1671095837
   ↳ fetched 1000 votes | total=8000 | next created_lt=1671076141
   ↳ fetched 603 votes | total=8603 | next created_lt=1671051621
🧾 votes 数：8603
✅ 已写入 8603 行 | 累计完成 proposals=117

🔍 [uniswapgovernance.eth] Proposal 118/189：[Temperature Check] Deploy Uniswap V3 on Scroll testnet
   ↳ fetched 1000 votes | total=1000 | next created_lt=1670073063
   ↳ fetched 1000 votes | total=2000 | next created_lt=1670045222
   ↳ fetched 1000 votes | total=3000 | next created_lt=1670007793
   ↳ fetched 1000 votes | total=4000 | next created_lt=1669988264
   ↳ fetched 1000 votes | total=5000 | next created_lt=1669975799
   ↳ fetched 1000 votes | total=6000 | next created_lt=1669966432
   ↳ fetched 1000 votes | total=7000 | next created_lt=1669958716
   ↳ fetched 1000 votes | total=8000 | next created_lt=1669949854
   ↳ fetched 1000 votes | total=9000 | next created_lt=16699

   ↳ fetched 477 votes | total=1477 | next created_lt=1649118418
🧾 votes 数：1477
✅ 已写入 1477 行 | 累计完成 proposals=136

🔍 [uniswapgovernance.eth] Proposal 137/189：[Temperature Check] Deploy Uniswap V3 on Moonbeam
   ↳ fetched 1000 votes | total=1000 | next created_lt=1648703465
   ↳ fetched 301 votes | total=1301 | next created_lt=1648682637
🧾 votes 数：1301
✅ 已写入 1301 行 | 累计完成 proposals=137

🔍 [uniswapgovernance.eth] Proposal 138/189：Temperature Check - Should Uniswap v3 be deployed to Gnosis Chain?
   ↳ fetched 909 votes | total=909 | next created_lt=1647955141
🧾 votes 数：909
✅ 已写入 909 行 | 累计完成 proposals=138

🔍 [uniswapgovernance.eth] Proposal 139/189：Consensus Check - Should Uniswap add the 1bp fee tier to Polygon?
   ↳ fetched 1000 votes | total=1000 | next created_lt=1647923852
   ↳ fetched 338 votes | total=1338 | next created_lt=1647903644
🧾 votes 数：1338
✅ 已写入 1338 行 | 累计完成 proposals=139

🔍 [uniswapgovernance.eth] Proposal 140/189：Temp Check - Should Uniswap add the 1bp fee tier to Poly


🔍 [uniswapgovernance.eth] Proposal 173/189：Consensus Check: Community-Enabled Analytics via Yield-Generating Investment Str
   ↳ fetched 330 votes | total=330 | next created_lt=1625440941
🧾 votes 数：330
✅ 已写入 330 行 | 累计完成 proposals=173

🔍 [uniswapgovernance.eth] Proposal 174/189：Temp Check: Larger Grant Construct // CEA + No Negative Net UNI
   ↳ fetched 464 votes | total=464 | next created_lt=1624388029
🧾 votes 数：464
✅ 已写入 464 行 | 累计完成 proposals=174

🔍 [uniswapgovernance.eth] Proposal 175/189：Temperature Check: Raise the proposal quorum threshold
   ↳ fetched 369 votes | total=369 | next created_lt=1624049742
🧾 votes 数：369
✅ 已写入 369 行 | 累计完成 proposals=175

🔍 [uniswapgovernance.eth] Proposal 176/189：Temperature Check - Managing Systemic Risk in Uniswap’s Community Treasury using
   ↳ fetched 408 votes | total=408 | next created_lt=1623693897
🧾 votes 数：408
✅ 已写入 408 行 | 累计完成 proposals=176

🔍 [uniswapgovernance.eth] Proposal 177/189：Consensus Check — Funding a Political Defense of DeFi 
